In [1]:
import pandas as pd
import numpy as np

In [2]:
# Code to read csv file into colaboratory:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)


#2. Get the file
downloaded = drive.CreateFile({'id':'1ytb-fcbDtFxL3Bkr_Ge7ycqqaS10s_yh'}) # replace the id with id of file you want to access
downloaded.GetContentFile('xyz.csv')  

#3. Read file as panda dataframe
import pandas as pd
xyz = pd.read_csv('xyz.csv') 

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,18,20,21,22,23,24,25,26,27,30,31,32,33,34,35,38) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
# Code to read csv file into colaboratory:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

#2. Get the file
downloaded = drive.CreateFile({'id':'1WPvGCyFoBcaqS9zPTd30HiONQSw00uei'}) # replace the id with id of file you want to access
downloaded.GetContentFile('Skyteam_Timetable.csv')  

#3. Read file as panda dataframe
import pandas as pd
pdf = pd.read_csv('Skyteam_Timetable.csv') 

In [4]:
data = xyz
data = data.drop('Unnamed: 0', 1)
data = data[['Last__Name', 'First__Name','PassengerMiddleName', 'BirthDate', 'Passenger_Sex', 'FamilyStatus', ''
      'PassengerDocument', 'TravelDoc', 'TicketNumber', 'NumberFlight', 'BookingCode', 'TravelClass', 'Fare',
      'DepartDate', 'DepartureTime', 'DepartureCountry', 'DepartureCity', 'AirportDeparture',
      'ArrivalDate', 'ArrivalTime', 'ArrivalCountry', 'CityArrival', 'ArrivalAirport',
      'uid', 'bonusprogramm', 'NickName', 'Status', 'ProgrammNumber',
      'Meal', 'Baggage_right', 'Baggage_left', 'Codeshare_not', 'CodeShare_tc', 'CodeShare_xl', 'Codeshare_json', 'Sequence', 
      'activity_type', 'CardsType',
       ]]
#Удалить Ячейки с пустой фамилией
data.dropna(subset=['Last__Name'], inplace=True)

In [5]:
#Удалить строки, где нет инфо о багаже
data['Baggage_right'].replace(' ', np.nan, inplace=True)
data.dropna(subset=['Baggage_right'], inplace=True)

In [8]:
pdf = pdf.drop('Unnamed: 0', 1)

In [9]:
pdf.rename(columns = {'Flight':'NumberFlight'}, inplace = True)

In [ ]:
del pdf['Validity']

In [16]:
pdf.drop_duplicates(subset ="NumberFlight", keep = False, inplace = True)

In [17]:
final_data = pd.merge(data, pdf, how ='left', on ='NumberFlight')
final_data

,Last__Name,First__Name,PassengerMiddleName,BirthDate,Passenger_Sex,FamilyStatus,PassengerDocument,TravelDoc,TicketNumber,NumberFlight,BookingCode,TravelClass,Fare,DepartDate,DepartureTime,DepartureCountry,DepartureCity,AirportDeparture,ArrivalDate,ArrivalTime,ArrivalCountry,CityArrival,ArrivalAirport,uid,bonusprogramm,NickName,Status,ProgrammNumber,Meal,Baggage_right,Baggage_left,Codeshare_not,CodeShare_tc,CodeShare_xl,Codeshare_json,Sequence,activity_type,CardsType,Days,Dep_Time,Arr_Time,Aircraft,Travel_Time,city_from,country_from,airport_from,city_to,country_to,airport_to
0,RUMIANTSEV,EGOR,EVGENEVICH,1972-04-01 00:00:00,Male,MR,7536 277407,7536 277407,9.76449e+15,SU1217,ENODVA,A,AFLXWG,2017-01-08,06:05,Russian Federation,SAMARA,KUF,2017-01-08,06:55,Russian Federation,Moscow,SVO,NaN,NaN,FrequentFlyer61148,Basic,SU 441106611,,2PC,Registered,NO,Own,Operated by Some Other Airline,False,2.0,NaN,NaN,1234567,06:05,06:55,32A,1H50M,Samara,Russian Federation,KU,Moscow,Russian Federation,SV
1,KOLTSOV,ARTUR,FILIPPOVICH,1976-12-15 00:00:00,Male,MR,0383 434647,0383 434647,9.8277e+14,SU1392,VSUZBT,Y,YRSTDA,2017-02-27,11:50,Russian Federation,MOSCOW,SVO,2017-02-27,16:00,Russian Federation,Perm,PEE,NaN,NaN,FrequentFlyer83796,Elite,DT 179843508,HNML,1PC,Delayed,NO,Own,Operated by Some Other Airline,False,97.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,PANINA,VLADA,IGOREVNA,1978-07-03 00:00:00,Female,MRS,5193 160929,5193 160929,5.30081e+15,SU1323,LKLXCE,Y,YRSTEZ,2017-01-24,02:15,Russian Federation,MURMANSK,MMK,2017-01-24,04:40,Russian Federation,Moscow,SVO,288850707.0,Aeroflot Bonus,FrequentFlyer75496,Elite+,SU 219805094,RVML,0PC,None,NO,Own,Operated by Some Other Airline,False,40.0,Flight,Airlines,1234567.1,02:15,04:40,32A.1,2H25M,Murmansk,Russian Federation,MM,Moscow,Russian Federation,SV
3,KISELEV,ZAHAR,TIMOFEEVICH,1997-10-01 00:00:00,Male,MR,6785 928243,6785 928243,7.45669e+15,SU1285,UXOMUJ,Y,YSTNBJ,2017-08-05,09:50,Russian Federation,KRASNODAR,KRR,2017-08-05,12:15,Russian Federation,Moscow,SVO,863183185.0,Flying Blue,FrequentFlyer61115,Elite+,FB 670556441,,1PC,None,NO,Own,Operated by Some Other Airline,False,53.0,Flight,Airlines,1234567,09:50,12:15,32A,2H25M,Krasnodar,Russian Federation,KRR,Moscow,Russian Federation,SV
4,BELKIN,EDUARD,BOGDANOVICH,1977-09-16 00:00:00,Male,MR,6164 103050,6164 103050,2.77477e+13,SU5611,HTMFRQ,Y,YFLXSZ,2017-01-26,08:00,Russian Federation,YUZHNO,UUS,2017-01-26,09:15,Russian Federation,Vladivostok,VVO,NaN,NaN,FrequentFlyer849,Elite,DT 371352280,,0PC,Registered,NO,Own,Operated by Some Other Airline,False,77.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85059,KULAKOVA,REGINA,IGOREVNA,NaN,NaN,NaN,NaN,3265 758746,1.5724e+15,SU1337,LNKSMY,A,AFLXNC,2017-01-08,21:10,NaN,NaN,ARH,2017-01-08,23:00,NaN,NaN,SVO,NaN,NaN,NaN,NaN,NaN,,0PC,NaN,NO,NaN,NaN,NaN,NaN,NaN,NaN,1234567,21:10,23:00,SU9,1H50M,Arkhangelsk,Russian Federation,ARH,Moscow,Russian Federation,SV
85060,ASTAHOVA,DARIA,ELISEEVNA,1983-08-15,NaN,NaN,NaN,5415 449602,1.95352e+15,SU1337,USUOKD,Y,YFLXBH,2017-01-08,21:10,NaN,NaN,ARH,2017-01-08,23:00,NaN,NaN,SVO,NaN,NaN,NaN,NaN,NaN,,0PC,NaN,NO,NaN,NaN,NaN,NaN,NaN,NaN,1234567,21:10,23:00,SU9,1H50M,Arkhangelsk,Russian Federation,ARH,Moscow,Russian Federation,SV
85061,HOHLOV,DANIEL,IAROMIROVICH,1991-07-18,NaN,NaN,NaN,6634 373755,5.40314e+15,SU1337,APTLVG,A,AFLXIK,2017-01-08,21:10,NaN,NaN,ARH,2017-01-08,23:00,NaN,NaN,SVO,NaN,NaN,NaN,NaN,NaN,,0PC,NaN,NO,NaN,NaN,NaN,NaN,NaN,NaN,1234567,21:10,23:00,SU9,1H50M,Arkhangelsk,Russian Federation,ARH,Moscow,Russian Federation,SV
85062,ZHAROVA,ALIIA,GLEBOVNA,NaN,NaN,NaN,NaN,1381 836595,2.27167e+15,SU1337,COIPAE,Y,YGRPYQ,2017-01-08,21:10,NaN,NaN,ARH,2017-01-08,23:00,NaN,NaN,SVO,NaN,NaN,NaN,NaN,NaN,,0PC,NaN,NO,NaN,NaN,NaN,NaN,NaN,NaN,1234567,21:10,23:00,SU9,1H50M,Arkhangel

In [18]:
final_data['Name'] = final_data['Last__Name']+' '+final_data['First__Name']+' '+final_data['PassengerMiddleName']
fly_count = final_data.groupby(['Name'])['Baggage_right'].apply(lambda x: len(x.unique()))
fly_count = pd.DataFrame(fly_count)

In [19]:
#Удалить те, где уникальной инфо о чемодане меньше 2
fly_count = fly_count.drop(fly_count[fly_count.Baggage_right < 2].index)
fly_count

,Baggage_right
Name,
ABRAMOV ANTON DEMIDOVICH,2
ABRAMOV ARTEM IGNATOVICH,2
ABRAMOV DANILA GEORGIEVICH,2
ABRAMOV EDUARD IGNATOVICH,2
ABRAMOV GEORGII ZAHAROVICH,2
...,...
ZYKOVA ULIANA IVANOVNA,2
ZYKOVA VALERIIA VALEREVNA,2
ZYKOVA VERONIKA OLEGOVNA,2


In [20]:
fly_count.reset_index(inplace = True)

In [21]:
#Оставить в исходных данных только те, где уникальная инфо о чемоданах больше 1
data_merge = pd.merge(fly_count, final_data, how ='inner', on ='Name')

In [22]:
data_merge = data_merge.drop('Days', 1)
data_merge = data_merge.drop('Sequence', 1)
data_merge = data_merge.drop('Codeshare_json', 1)
data_merge = data_merge.drop('CodeShare_xl', 1)
data_merge = data_merge.drop('CodeShare_tc', 1)
data_merge = data_merge.drop('Codeshare_not', 1)
data_merge = data_merge.drop('PassengerDocument', 1)
data_merge = data_merge.drop('uid', 1)

In [23]:
data_merge['DepartureCity'] = data_merge['DepartureCity'].astype('string')
data_merge['DepartureCountry'] = data_merge['DepartureCountry'].astype('string')
data_merge['ArrivalCountry'] = data_merge['ArrivalCountry'].astype('string')
data_merge['CityArrival'] = data_merge['CityArrival'].astype('string')

In [24]:
 #Заполнение города отправления
 for i in range(len(data_merge)):
    if data_merge.iloc[i].isna()['DepartureCity'] == True:
      data_merge.at[i, 'DepartureCity'] = data_merge.iloc[i]['city_from']

In [ ]:
 #Заполнение страны отправления и города и страны прибытия
for i in range(len(data_merge)):
  if data_merge.iloc[i].isna()['DepartureCountry'] == True:
    data_merge.at[i, 'DepartureCountry'] = data_merge.iloc[i]['country_from']
  if data_merge.iloc[i].isna()['ArrivalCountry'] == True:
    data_merge.at[i, 'ArrivalCountry'] = data_merge.iloc[i]['country_to']
  if data_merge.iloc[i].isna()['CityArrival'] == True:
    data_merge.at[i, 'CityArrival'] = data_merge.iloc[i]['city_to']
 

In [26]:
data_merge = data_merge.drop('city_from', 1)
data_merge = data_merge.drop('country_from', 1)
data_merge = data_merge.drop('country_to', 1)
data_merge = data_merge.drop('city_to', 1)

In [27]:
#Удалить дублирование по Номеру билета
data_merge.drop_duplicates(subset ='TicketNumber',

                     keep = False, inplace = True)

In [28]:
data_merge['DepartDate'] = data_merge['DepartDate'].astype('datetime64')
data_merge['ArrivalDate'] = data_merge['ArrivalDate'].astype('datetime64')

In [29]:
#Создать столбец с признаком Проверки
data_merge['Flag'] = False

In [30]:
#Удалить столбцы, где город отправления и город прибытия пустые
data_merge.dropna(subset=['DepartureCity'], inplace=True)
data_merge.dropna(subset=['CityArrival'], inplace=True)

In [31]:
data_merge

,Name,Baggage_right_x,Last__Name,First__Name,PassengerMiddleName,BirthDate,Passenger_Sex,FamilyStatus,TravelDoc,TicketNumber,NumberFlight,BookingCode,TravelClass,Fare,DepartDate,DepartureTime,DepartureCountry,DepartureCity,AirportDeparture,ArrivalDate,ArrivalTime,ArrivalCountry,CityArrival,ArrivalAirport,bonusprogramm,NickName,Status,ProgrammNumber,Meal,Baggage_right_y,Baggage_left,activity_type,CardsType,Dep_Time,Arr_Time,Aircraft,Travel_Time,airport_from,airport_to,Flag
2,ABRAMOV ANTON DEMIDOVICH,2,ABRAMOV,ANTON,DEMIDOVICH,NaN,NaN,NaN,7143 491318,7.63981e+15,SU1172,VSWCWR,Y,YRSTPZ,2017-10-26,21:25,Russian Federation,Moscow,SVO,2017-10-27,00:50,Russian Federation,Astrakhan,ASF,NaN,NaN,NaN,NaN,,1PC,NaN,NaN,NaN,22:25,01:50+1,73H,2H25M,SV,AS,False
3,ABRAMOV ARTEM IGNATOVICH,2,ABRAMOV,ARTEM,IGNATOVICH,1990-07-04 00:00:00,Male,MR,1306 969617,7.25021e+15,SU1355,OMDLVH,Y,YFLXLP,2017-04-20,21:05,<NA>,VORONEZH,VOZ,2017-04-20,22:20,<NA>,Moscow,SVO,NaN,NaN,NaN,NaN,,1PC,Registered,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
4,ABRAMOV ARTEM IGNATOVICH,2,ABRAMOV,ARTEM,IGNATOVICH,07/04/1990,Male,MR,1306 969617,6.77377e+15,SU1350,HEOTUD,A,AFLXVQ,2017-03-04,08:00,<NA>,MOSCOW,SVO,2017-03-04,09:20,<NA>,Voronezh,VOZ,NaN,NaN,NaN,NaN,,2PC,Delayed,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
5,ABRAMOV ARTEM IGNATOVICH,2,ABRAMOV,ARTEM,IGNATOVICH,1990-07-04,NaN,MR,1306 969617,1.36287e+15,SU1106,LDLLVQ,Y,YFLXNN,2017-02-01,16:30,<NA>,MOSCOW,SVO,2017-02-01,18:50,<NA>,KRASNODAR,KRR,NaN,NaN,NaN,NaN,,2PC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
17,ABRAMOV IAN PLATONOVICH,2,ABRAMOV,IAN,PLATONOVICH,01/05/1973,Male,MR,6159 807770,8.49931e+15,SU1375,VCMTVY,Y,YSTNWU,2017-01-01,18:55,<NA>,BELGOROD,EGO,2017-01-01,20:20,<NA>,Moscow,SVO,NaN,NaN,NaN,NaN,,1PC,Delayed,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39073,ZYKOVA VERONIKA OLEGOVNA,2,ZYKOVA,VERONIKA,OLEGOVNA,1974-09-20,NaN,NaN,5058 850845,7.83758e+15,SU1457,JBRXHB,Y,YSTNDS,2017-02-01,20:55,Russian Federation,Kemerovo,KEJ,2017-02-01,21:25,Russian Federation,Moscow,SVO,NaN,NaN,NaN,NaN,,2PC,NaN,NaN,NaN,20:55,21:25,EQV,4H30M,KEJ,SV,False
39075,ZYKOVA VITALINA PAVLOVNA,2,ZYKOVA,VITALINA,PAVLOVNA,1972-04-28,NaN,MRS,1169 416233,1.15212e+15,SU1183,TAJDMM,Y,YSTNYR,2017-02-28,18:20,<NA>,VOLGOGRAD,VOG,2017-02-28,19:20,<NA>,MOSCOW,SVO,NaN,NaN,NaN,NaN,LCML,2PC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
39077,ZYKOVA VLADA BOGDANOVNA,3,ZYKOVA,VLADA,BOGDANOVNA,NaN,NaN,MRS,6767 548385,8.41298e+14,SU1352,HCHHVT,Y,YSTNDE,2017-12-15,22:50,<NA>,MOSCOW,SVO,2017-12-16,00:10,<NA>,VORONEZH,VOZ,Delta SkyMiles,NaN,NaN,DT 209991132,,2PC,NaN,Flight,Airlines,NaN,NaN,NaN,NaN,NaN,NaN,False
39078,ZYKOVA VLADA BOGDANOVNA,3,ZYKOVA,VLADA,BOGDANOVNA,NaN,NaN,MRS,6767 548385,1.34824e+15,SU1193,ANHZQX,Y,YFLXRU,2017-11-10,18:05,<NA>,KAZAN,KZN,2017-11-10,19:55,<NA>,MOSCOW,SVO,NaN,NaN,NaN,NaN,MOML,0PC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False


In [32]:
data_merge = data_merge.sort_values(by=['Name'])

In [38]:
#Поиск подозрительных пассажирова
#Увеличиваем шаг на 10, так как макисмальное количество записей по одному пассажиру 10
for i in range(11416):
  for j in range(i, i+10):
    if data_merge.iloc[i]['Name'] == data_merge.iloc[j]['Name'] and data_merge.iloc[i]['Baggage_right_y'] != data_merge.iloc[j]['Baggage_right_y'] and (data_merge.iloc[i]['CityArrival'] == data_merge.iloc[j]['DepartureCity'] or data_merge.iloc[j]['CityArrival'] == data_merge.iloc[i]['DepartureCity']) and abs(((data_merge.iloc[i]['ArrivalDate']-data_merge.iloc[j]['DepartDate']).days)) <= 6:
      data_merge.loc[i,'Flag'] = True

In [40]:
itog = data_merge[data_merge['Flag']==True]

In [ ]:
itog.dropna(subset=['Name'], inplace=True)
itog = itog.drop(['Name', 'Baggage_right_x'], 1)

In [55]:
itog

,Last__Name,First__Name,PassengerMiddleName,BirthDate,Passenger_Sex,FamilyStatus,TravelDoc,TicketNumber,NumberFlight,BookingCode,TravelClass,Fare,DepartDate,DepartureTime,DepartureCountry,DepartureCity,AirportDeparture,ArrivalDate,ArrivalTime,ArrivalCountry,CityArrival,ArrivalAirport,bonusprogramm,NickName,Status,ProgrammNumber,Meal,Baggage_right_y,Baggage_left,activity_type,CardsType,Dep_Time,Arr_Time,Aircraft,Travel_Time,airport_from,airport_to,Flag
82,ABRAMOVA,KIRA,RADMIROVNA,1978-12-02,NaN,MRS,4304 139014,3.37548e+15,SU1397,NVEOTD,A,AGRPQH,2017-04-25,06:05,<NA>,SYKTYVKAR,SCW,2017-04-25,08:05,<NA>,MOSCOW,SVO,NaN,NaN,NaN,NaN,,0PC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
244,AGAPOV,ALBERT,STANISLAVOVICH,1998-03-01,Male,MR,4298 956965,8.04556e+13,SU1758,DTOTPO,Y,YRSTMX,2017-02-28,07:55,Russian Federation,MOSCOW,SVO,2017-02-28,10:40,Russian Federation,VOLGOGRAD,VOG,NaN,FrequentFlyer29828,Elite+,KE 486653809,,2PC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
249,AGAPOV,ARTUR,DEMIDOVICH,1973-07-10,NaN,MR,59 7987861,7.31467e+15,SU1461,TIFFRU,Y,YFLXPF,2017-01-15,18:30,<NA>,NOVOSIBIRSK,OVB,2017-01-15,19:00,<NA>,MOSCOW,SVO,NaN,NaN,NaN,NaN,,1PC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
265,AGAPOV,EGOR,RADMIROVICH,NaN,NaN,NaN,3851 126588,1.9453e+15,SU1547,MVDPHC,A,ARSTKB,2017-04-09,11:35,Russian Federation,Anapa,AAQ,2017-04-09,13:50,Russian Federation,Moscow,SVO,NaN,NaN,NaN,NaN,,2PC,NaN,NaN,NaN,11:35,13:50,32A,2H15M,AA,SV,True
271,AGAPOV,PLATON,ALEKSEEVICH,NaN,NaN,MR,21 7954626,5.9799e+15,SU1279,DYCPYN,J,JRSTWL,2017-04-26,07:00,<NA>,KAZAN,KZN,2017-04-26,08:35,<NA>,MOSCOW,SVO,NaN,NaN,NaN,NaN,,1PC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11100,GRIBOVA,ALIIA,GRIGOREVNA,1996-05-20 00:00:00,Female,NaN,9260 193111,9.52182e+15,SU1511,YFMLEY,Y,YGRPFJ,2017-10-17,03:30,Russian Federation,Surgut,SGC,2017-10-17,04:50,Russian Federation,Moscow,SVO,NaN,NaN,NaN,NaN,STML,1PC,None,NaN,NaN,04:30,05:50,32A,3H20M,SG,SV,True
11209,GRISHIN,GERMAN,ODISSEICH,1974-08-27,NaN,NaN,8629 070280,4.37339e+15,SU1217,JGZKSV,Y,YGRPAI,2017-07-09,06:05,Russian Federation,Samara,KUF,2017-07-09,06:55,Russian Federation,Moscow,SVO,NaN,NaN,NaN,NaN,,2PC,NaN,NaN,NaN,06:05,06:55,32A,1H50M,KU,SV,True
11234,GRISIN,GEORGII,VITALEVICH,01/17/1999,Male,MR,2367 926785,4.24246e+15,SU1362,XTSWGB,Y,YRSTQB,2017-03-07,16:50,<NA>,MOSCOW,SVO,2017-03-07,19:35,<NA>,Saratov,RTW,NaN,NaN,NaN,NaN,VGML,2PC,Transit,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
11340,GUBANOV,MIROSLAV,ILDAROVICH,1984-06-05 00:00:00,Male,NaN,7379 414545,6.54786e+15,SU5620,VXLYOV,Y,YGRPRS,2017-03-21,09:50,Russian Federation,Khabarovsk,KHV,2017-03-21,12:25,Russian Federation,Yuzhno,UUS,NaN,NaN,NaN,NaN,ORML,0PC,None,NaN,NaN,09:50,12:25,DH4,2H35M,KHV,UUS,True


In [51]:
itog.to_csv('/content/drive/MyDrive/Багаж.csv')


In [50]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
